In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Base directory to save everything
base_dir = "/content/drive/MyDrive/BERT_Precision_Optimized"
os.makedirs(base_dir, exist_ok=True)

print(f"✅ All files will be saved to: {base_dir}")

Mounted at /content/drive
✅ All files will be saved to: /content/drive/MyDrive/BERT_Precision_Optimized


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv("/content/Total_dataset.csv")
df = df.dropna(subset=['User Comment', 'Label'])
df['Label'] = df['Label'].astype(int)

# Balance dataset (equal 0 and 1)
class_0 = df[df['Label'] == 0]
class_1 = df[df['Label'] == 1]
min_class_size = min(len(class_0), len(class_1))

balanced_df = pd.concat([
    class_0.sample(min_class_size, random_state=42),
    class_1.sample(min_class_size, random_state=42)
])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_df, test_df = train_test_split(
    balanced_df,
    test_size=0.2,
    random_state=42,
    stratify=balanced_df['Label']
)

train_df.to_csv(f"{base_dir}/balanced_train_set.csv", index=False)
test_df.to_csv(f"{base_dir}/balanced_test_set.csv", index=False)

print(f"✅ Balanced train and test sets saved to {base_dir}")

✅ Balanced train and test sets saved to /content/drive/MyDrive/BERT_Precision_Optimized


In [ ]:
!pip install transformers datasets

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Use BERT or optionally switch to DeBERTa
model_name = "bert-base-uncased"
# model_name = "microsoft/deberta-v3-base"  # Uncomment to try DeBERTa

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Model '{model_name}' loaded and ready.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model 'bert-base-uncased' loaded and ready.


In [ ]:
import datasets

train_df = pd.read_csv(f"{base_dir}/balanced_train_set.csv")
test_df = pd.read_csv(f"{base_dir}/balanced_test_set.csv")

def preprocess_function(examples):
    texts = [
        f"User Comment: {uc} Author Reply: {ar}"
        for uc, ar in zip(examples["User Comment"], examples["Author Reply"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

train_dataset = datasets.Dataset.from_pandas(train_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(preprocess_function, batched=True, batch_size=16)
test_dataset = test_dataset.map(preprocess_function, batched=True, batch_size=16)

train_dataset = train_dataset.rename_column("Label", "labels")
test_dataset = test_dataset.rename_column("Label", "labels")

train_dataset = train_dataset.remove_columns(["User Comment", "Author Reply"])
test_dataset = test_dataset.remove_columns(["User Comment", "Author Reply"])

print("✅ Datasets prepared for training.")

Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

Map:   0%|          | 0/272 [00:00<?, ? examples/s]

✅ Datasets prepared for training.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_df['Label'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

print(f"✅ Computed class weights: {class_weights.cpu().numpy()}")

✅ Computed class weights: [1. 1.]


In [ ]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

training_args = TrainingArguments(
    output_dir=f"{base_dir}/bert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir=f"{base_dir}/logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions),
        "recall": recall_score(labels, predictions),
        "f1": f1_score(labels, predictions),
    }

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("✅ Trainer ready.")

✅ Trainer ready.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

trainer.save_model(f"{base_dir}/bert_finetuned_model")
tokenizer.save_pretrained(f"{base_dir}/bert_finetuned_model")

print(f"✅ Model saved to {base_dir}/bert_finetuned_model")

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.663200,0.671903,0.569853,0.542222,0.897059,0.675900
2,0.608100,0.593178,0.687500,0.671141,0.735294,0.701754
3,0.410800,0.756563,0.665441,0.671756,0.647059,0.659176
4,0.247600,0.893039,0.658088,0.661654,0.647059,0.654275
5,0.128400,1.393599,0.680147,0.662252,0.735294,0.696864
6,0.021500,1.784443,0.669118,0.649351,0.735294,0.689655
7,0.017700,2.007117,0.636029,0.639098,0.625000,0.631970
8,0.001400,2.116087,0.643382,0.627451,0.705882,0.664360
9,0.001000,2.175643,0.643382,0.640288,0.654412,0.647273
10,0.001000,2.222206,0.628676,0.631579,0.617647,0.624535


✅ Model saved to /content/drive/MyDrive/BERT_Precision_Optimized/bert_finetuned_model


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def batch_predict_with_threshold(dataset, batch_size=8, threshold=0.6):
    pred_labels = []
    for i in range(0, len(dataset), batch_size):
        batch = dataset.iloc[i:i+batch_size]
        texts = [
            f"User Comment: {row['User Comment']} Author Reply: {row['Author Reply']}"
            for _, row in batch.iterrows()
        ]
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            preds = [1 if p[1] > threshold else 0 for p in probs]

        pred_labels.extend(preds)

    return pred_labels

# ----------------------
# Predict & Evaluate on Training Set
# ----------------------
train_df["predicted_label"] = batch_predict_with_threshold(train_df)

# Save training predictions
train_df.to_csv(f"{base_dir}/train_predictions_with_threshold.csv", index=False)

# Calculate metrics for training set
train_accuracy = accuracy_score(train_df["Label"], train_df["predicted_label"])
train_precision = precision_score(train_df["Label"], train_df["predicted_label"])
train_recall = recall_score(train_df["Label"], train_df["predicted_label"])
train_f1 = f1_score(train_df["Label"], train_df["predicted_label"])

print("\n✅ Training Set Metrics (with Threshold)")
print(f"Train Accuracy: {train_accuracy:.2%}")
print(f"Train Precision: {train_precision:.2%}")
print(f"Train Recall: {train_recall:.2%}")
print(f"Train F1 Score: {train_f1:.2%}")

# ----------------------
# Predict & Evaluate on Test Set
# ----------------------
test_df["predicted_label"] = batch_predict_with_threshold(test_df)

# Save test predictions
test_df.to_csv(f"{base_dir}/test_predictions_with_threshold.csv", index=False)

# Calculate metrics for test set
test_accuracy = accuracy_score(test_df["Label"], test_df["predicted_label"])
test_precision = precision_score(test_df["Label"], test_df["predicted_label"])
test_recall = recall_score(test_df["Label"], test_df["predicted_label"])
test_f1 = f1_score(test_df["Label"], test_df["predicted_label"])

print("\n✅ Final Test Set Metrics (with Threshold)")
print(f"Test Accuracy: {test_accuracy:.2%}")
print(f"Test Precision: {test_precision:.2%}")
print(f"Test Recall: {test_recall:.2%}")
print(f"Test F1 Score: {test_f1:.2%}")

print(f"\n✅ Predictions saved to {base_dir}/train_predictions_with_threshold.csv and {base_dir}/test_predictions_with_threshold.csv")


✅ Training Set Metrics (with Threshold)
Train Accuracy: 79.98%
Train Precision: 83.23%
Train Recall: 75.09%
Train F1 Score: 78.95%

✅ Final Test Set Metrics (with Threshold)
Test Accuracy: 67.28%
Test Precision: 70.80%
Test Recall: 58.82%
Test F1 Score: 64.26%

✅ Predictions saved to /content/drive/MyDrive/BERT_Precision_Optimized/train_predictions_with_threshold.csv and /content/drive/MyDrive/BERT_Precision_Optimized/test_predictions_with_threshold.csv
